In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from IPython.display import display, HTML

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer in 4-bit precision
model_name = "mistralai/Mistral-7B-v0.1"  # Replace with the correct path if locally available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",            # Automatically assigns layers to devices
    load_in_4bit=True,             # Enable 4-bit loading
    torch_dtype=torch.float16,     # Mixed precision for additional VRAM savings
)

# Summarization function with deterministic output
def summarize_text(input_text, max_length=1000, temperature=0.1):
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    summary_ids = model.generate(
        inputs["input_ids"], 
        max_length=max_length, 
        temperature=temperature, 
        do_sample=False
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Example usage
# input_text = '''
# Summarize the text: In telecommunications, orthogonal frequency-division multiplexing (OFDM) is a type 
# of digital transmission used in digital modulation for encoding digital (binary) 
# data on multiple carrier frequencies. OFDM has developed into a popular scheme for 
# wideband digital communication, used in applications such as digital television and 
# audio broadcasting, DSL internet access, wireless networks, power line networks, 
# and 4G/5G mobile communications.[1] OFDM is a frequency-division multiplexing (FDM) 
# scheme that was introduced by Robert W. Chang of Bell Labs in 1966.[2][3][4] In OFDM, 
# the incoming bitstream representing the data to be sent is divided into multiple streams. 
# Multiple closely spaced orthogonal subcarrier signals with overlapping spectra are 
# transmitted, with each carrier modulated with bits from the incoming stream so multiple 
# bits are being transmitted in parallel.[5] Demodulation is based on fast Fourier transform 
# algorithms. OFDM was improved by Weinstein and Ebert in 1971 with the introduction of 
# a guard interval, providing better orthogonality in transmission channels affected by 
# multipath propagation.[6] Each subcarrier (signal) is modulated with a conventional modulation 
# scheme (such as quadrature amplitude modulation or phase-shift keying) at a low symbol 
# rate. This maintains total data rates similar to conventional single-carrier modulation 
# schemes in the same bandwidth.'''.replace("\n", "")
input_text = 'good morning'
summary = summarize_text(input_text)
display(HTML(f"<b>Summary:</b> {summary}"))

D:\Anaconda\envs\llm\lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
)

# Load the model with the quantization configuration
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# Define the prompt
prompt = """
Summarize the text: In telecommunications, orthogonal frequency-division multiplexing (OFDM) 
is a type of digital transmission used in digital modulation for encoding digital (binary) data
on multiple carrier frequencies. OFDM has developed into a popular scheme for wideband digital 
communication, used in applications such as digital television and audio broadcasting, DSL internet 
access, wireless networks, power line networks, and 4G/5G mobile communications.
""".replace("\n", "")

# Tokenize the prompt and move to CUDA
model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

# Generate text
generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)

# Decode the generated text
output_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(output_text)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Summarize the text: In telecommunications, orthogonal frequency-division multiplexing (OFDM) is a type of digital transmission used in digital modulation for encoding digital (binary) dataon multiple carrier frequencies. OFDM has developed into a popular scheme for wideband digital communication, used in applications such as digital television and audio broadcasting, DSL internet access, wireless networks, power line networks, and 4G/5G mobile communications. The orthogonal frequencies are widely spaced enough that a frequency-selective channel distortion affects each channel independently. The signal can be recovered by demodulating on the same carrier, as orthogonality is maintained. Because a frequency-selective channel changes the amplitude of each subcarrier's signal differently, inter-channel interference is avoided. The frequency selectivity is achieved using a low roll-off factor on a discrete…
